In [ ]:
%load_ext autoreload
%autoreload 2
import os
project_name = "PaddleSeg"
os.chdir(os.path.join(os.getcwd().split(project_name, 1)[0], project_name))

In [ ]:
dir_path = os.path.join('Z:', 'datasets', 'cracks', 'Updated_Dataset', 'Final_Dataset')

for prefix in ['train', 'test']:
  with open(os.path.join(dir_path, f'{prefix}_main.txt'), 'w', encoding='utf-8') as f_main:
    with open(os.path.join(dir_path, f'{prefix}.txt'), 'r', encoding='utf-8') as f:
      for i, l in enumerate(f.readlines()):
        name = l.strip()
        image_name = os.path.join('Images', name)
        label_name = os.path.join('Masks', name)
        image_exists = os.path.exists(os.path.join(dir_path, image_name))
        label_exists = os.path.exists(os.path.join(dir_path, label_name))
        if image_exists and label_exists:
          f_main.write(f"{image_name} {label_name}\n")

In [ ]:
base_dir = 'Z:\datasets\cracks\kzm\paddle'
with open(os.path.join(base_dir, 'test.txt'), 'w', encoding='utf-8') as ftest:
  with open(os.path.join(base_dir, 'train.txt'), 'w', encoding='utf-8') as ftrain:
    for i, fname in enumerate(os.listdir(os.path.join(base_dir, 'images'))):
      fname = fname.split('.')[0]

      image_name = [a for a in os.listdir(os.path.join(base_dir, 'images')) if a.startswith(fname)][0]
      label_name = [a for a in os.listdir(os.path.join(base_dir, 'annotations')) if a.startswith(fname)][0]
      image_path = os.path.join(base_dir, image_name)
      label_path = os.path.join(base_dir, label_name)
      output = f'{image_path} {label_path}\n'
      if i > 2:
        ftrain.write(output)
      else:
        ftest.write(output)

In [ ]:
dir_path = os.path.join('Z:', 'datasets', 'cracks', 'Updated_Dataset', 'Final_Dataset')
with open(os.path.join(dir_path, 'train_second.txt'), 'w', encoding='utf-8') as f2:
  with open(os.path.join(dir_path, 'train_main.txt'), 'r', encoding='utf-8') as f:
      lines = f.readlines()
      print(len(lines))
      for l in lines:
          # Splitting the line and stripping the newline character
          img_path, mask_path = [s.strip() for s in l.split(' ')]
          if os.path.exists(os.path.join(dir_path, img_path)) and os.path.exists(os.path.join(dir_path, mask_path)):
              output = f'{mask_path} {img_path}\n'
              f2.write(output)

In [ ]:
from PIL import Image
import numpy as np

def count_unique_pixels(image_path):
    # Load the image
    image = Image.open(image_path)

    # Convert to grayscale if it's a color image
    if image.mode != 'L':
        image = image.convert('L')

    # Convert to NumPy array for efficient computation
    image_array = np.array(image)

    # Find unique pixel values and their counts
    unique_pixels, counts = np.unique(image_array, return_counts=True)

    # Creating a dictionary for pixel values and their counts
    pixel_count = dict(zip(unique_pixels, counts))

    return pixel_count

# Example usage
image_path = os.path.join(f'P:\PaddleSeg\outputs\hrsegnetb48\\result\\2\pseudo_color_prediction\\2_5.png')
# image_path = os.path.join('Z:\datasets\cracks\kzm\masks', '0_9.png')
unique_pixel_count = count_unique_pixels(image_path)
print(unique_pixel_count)


In [ ]:
dir_path = os.path.join('Z:', 'datasets', 'cracks', 'Updated_Dataset', 'Final_Dataset')

valid_images = []

for fname in os.listdir(os.path.join(dir_path, 'Images')):
  if os.path.exists(os.path.join(dir_path, 'FIXED', fname)):
    valid_images.append(fname)

from collections import defaultdict

def group_strings_by_prefix(strings):
    prefix_groups = defaultdict(list)
    
    # Iterate over each string
    for string in strings:
        # Use the first three characters as the key
        prefix = string[:3]
        # Append the string to the corresponding group
        prefix_groups[prefix].append(string)
    
    # Convert defaultdict to a regular dictionary for output
    return dict(prefix_groups)


dicted = group_strings_by_prefix(valid_images)

In [ ]:
import random

def split_dataset(strings, train_ratio, val_ratio):
    # Calculate total number of items based on the ratio
    total_ratio = train_ratio + val_ratio
    train_count = len(strings) * train_ratio // total_ratio

    # Shuffle the list in-place to ensure random distribution
    random.shuffle(strings)

    # Split the list
    train_subset = strings[:train_count]
    val_subset = strings[train_count:]

    return train_subset, val_subset

In [54]:
dicted.keys()

dict_keys(['Ris', 'non', 'GAP', 'CRA'])

In [55]:
with open(os.path.join(dir_path, 'train_main.txt'), 'w', encoding='utf-8') as train_f:
  with open(os.path.join(dir_path, 'val_main.txt'), 'w', encoding='utf-8') as val_f:
    for key, img_names in dicted.items():
      if key == 'non':
        continue
      train_ds, val_ds = split_dataset(img_names, 8, 2)
      print(key, len(img_names), len(train_ds), len(val_ds), sum([len(train_ds), len(val_ds)]))
      for fname in train_ds:
        image_path = os.path.join('Images', fname)
        mask_path = os.path.join('FIXED', fname)
        train_f.write(f'{image_path} {mask_path}\n')
      for fname in val_ds:
        image_path = os.path.join('Images', fname)
        mask_path = os.path.join('FIXED', fname)
        val_f.write(f'{image_path} {mask_path}\n')


Ris 2736 2188 548 2736
GAP 383 306 77 383
CRA 1785 1428 357 1785


In [ ]:
with open(os.path.join(dir_path, 'train_main.txt'), 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for l in lines:
        # Splitting the line and stripping the newline character
        img_path, mask_path = [s.strip() for s in l.split(' ')]
        if os.path.exists(os.path.join(dir_path, img_path)) and os.path.exists(os.path.join(dir_path, mask_path)):
            output = f'{mask_path} {img_path}\n'
            f2.write(output)

In [65]:
from PIL import Image
import numpy as np

dir_path = os.path.join('Z:', 'datasets', 'cracks', 'Updated_Dataset', 'Final_Dataset')

def binarize_image(file_name, threshold=127):
    img_path = os.path.join(dir_path, 'Masks', file_name)
    image = Image.open(img_path)
    image_array = np.array(image)
    binarized_array = np.where(image_array > threshold, 255, 0)
    binarized_image = Image.fromarray(binarized_array.astype(np.uint8))
    binarized_image.save(os.path.join(dir_path, 'Fixed', file_name))

with open(os.path.join(dir_path, 'val_main.txt'), 'r', encoding='utf-8') as f:
  lines = f.readlines()
  print(len(lines))
  for l in lines:
    # Splitting the line and stripping the newline character
    img_path, mask_path = [s.strip() for s in l.split(' ')]
    fi, sec = img_path.split(os.path.sep)
    img_full_path = os.path.join(dir_path, img_path)
    mask_full_path = os.path.join(dir_path, 'Masks', sec)
    # print(f'[{img_full_path}]')
    # print(f'[{mask_full_path}]')
    if os.path.exists(img_full_path) and os.path.exists(mask_full_path):
      _, file_name = mask_path.split(os.sep)
      # print(mask_path)
      binarize_image(file_name)

982


In [ ]:
def change_colors(file_name):
    img_path = os.path.join(dir_path, 'Masks', file_name)
    image = Image.open(img_path)

    # Ensure the image is in RGBA mode
    if image.mode != 'RGBA':
        image = image.convert('RGBA')
    
    # Convert image to numpy array
    data = np.array(image)

    # Create a mask for non-black (all non-zero) pixels - these will become red
    non_black_pixels_mask = (data[:, :, :3] != 0).any(-1)

    # Create a mask for black (all zero) pixels - these will become green
    black_pixels_mask = (data[:, :, :3] == 0).all(-1)

    # Change non-black (white and other colors) to red
    data[black_pixels_mask] = [255, 0, 0, 255]  # RGBA for red

    # Change black to green
    data[non_black_pixels_mask] = [0, 255, 0, 255]  # RGBA for green

    # Convert the numpy array back to an image
    new_image = Image.fromarray(data)

    # Save the new image
    new_image.save(os.path.join(dir_path, 'Fixed2', file_name))

with open(os.path.join(dir_path, 'val_main.txt'), 'r', encoding='utf-8') as f:
  lines = f.readlines()
  for l in lines:
    # Splitting the line and stripping the newline character
    img_path, mask_path = [s.strip() for s in l.split(' ')]
    mask_full_path = os.path.join(dir_path, mask_path)
    img_full_path = os.path.join(dir_path, img_path)
    if os.path.exists(img_full_path) and os.path.exists(mask_full_path):
      _, file_name = mask_path.split(os.sep)
      # print(mask_path)
      change_colors(file_name)